In [1]:
import pickle
import csv
import timeit
import random

In [2]:
#TF-IDF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [3]:
csv_file_path = "DataSet/data.csv"
    
tfidf_vectorizer_pickle_path = "tfidf_vectorizer.pickle"
tfidf_matrix_train_pickle_path = "tfidf_matrix_train.pickle"

In [9]:
def load_csv():
    #load dataset data into sentences array
    with open(csv_file_path, "r") as sentences_on_file:
        reader = csv.reader(sentences_on_file, delimiter = ",")
        for row in reader:
            sentences.append(row[0])
            i += 1
        print (sentences)

In [10]:
def talk_to_bot_primary(test_sentence):
    i=0
    #array to store dataset values row-wise 
    sentences = []
    #enter test sentence
    test_set = (test_sentence, "")

    #for indexes
    sentences.append(" No You.") 
    sentences.append(" No You.")

    try:
        ##------------ to use ---------------##
        # first load dimension space
        f = open(tfidf_vectorizer_pickle_path, 'rb')
        tfidf_vectorizer = pickle.load(f)
        f.close()

        # then load tf-idf of dataset
        f = open(tfidf_matrix_train_pickle_path, 'rb')
        tfidf_matrix_train = pickle.load(f)
        f.close()
        # ----------------------------------------#
    except:
        ##------------ to train -------------##
        #variable to see how much time it takes to train
        start = timeit.default_timer()
        
        load_csv(csv_file_path)

        # now we would start on with training, create a tfidfvectorizer object
        tfidf_vectorizer = TfidfVectorizer()

        # 1- identify dimension of the space
        # 2- learn tf-idf of the dataset, both actions are performed here
        tfidf_matrix_train = tfidf_vectorizer.fit_transform(sentences)

        # record time that it finished training
        stop = timeit.default_timer()
        print ("training time took was : ")
        print (stop - start)

        # then we would save these 2 objs (dimension space and tf-idf to temp files)
        # we use pickle lib

        # first save dimension space
        f = open(tfidf_vectorizer_pickle_path, 'wb')
        pickle.dump(tfidf_vectorizer, f)
        f.close()

        # then save tf-idf of dataset
        f = open(tfidf_matrix_train_pickle_path, 'wb')
        pickle.dump(tfidf_matrix_train, f)
        f.close()
        # -----------------------------------------#
    
    # use the learnt dimension space
    # to run TF-IDF on the query
    tfidf_matrix_test = tfidf_vectorizer.transform(test_set)

    # then run cosine similarity between the 2 tf-idfs
    cosine = cosine_similarity(tfidf_matrix_test, tfidf_matrix_train)
    cosine = np.delete(cosine, 0)
    
    # then get the max score
    max = cosine.max()
    response_index = 0
  
  
    # if score is more than 0.7
    if (max > 0.7): 
        # we can afford to get multiple high score documents to choose from
        new_max = max - 0.01

        # load them to a list
        list = np.where(cosine > new_max)

        # choose a random one to return to the user 
        # this happens to make bot answers differently to same sentence
        response_index = random.choice(list[0])

    else:
        # else we would simply return the highest score
        response_index = np.where(cosine == max)[0][0] + 2 
       

    j = 0

    # loop to return the next cell on the row , ( the response cell )
    with open(csv_file_path, "r") as sentences_file:
        reader = csv.reader(sentences_file, delimiter=',')
        for row in reader:
            j += 1  # we begin with 1 not 0 &    j is initialized by 0
            if j == response_index:
                return row[1], response_index,
                break

In [ ]:
while 1:
    sentence = input("talk to bot : ")

    response_primary, line_id_primary = talk_to_bot_primary(sentence)
    print (response_primary)
    print

talk to bot : what is your name
my name is Lina, but you can call me lina 
talk to bot : how are you? 
i'm fine !  did you rest well ?
talk to bot : no
: d yo le tengo miedo a los cordones de las zapatillas sin anudar.   pueden ser peligrosos !
talk to bot : it is bad
ow  why is that ?  if you what to share
talk to bot : i am ired
good !   do you speak english ?
talk to bot : yes
do you want to listen to some music i wrote ?  
talk to bot : maybe
do you have other opinion ?
talk to bot : no
 hahaha
